In [1]:
import numpy as np
import open3d as o3d
import os
import pickle
import copy
import time

from pcd_obs_env import PCDObsEnv
from segmentation import BackgroundGeometry
from object_registration import preprocess_point_cloud, execute_global_registration, draw_registration_result

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def display_inlier_outlier(cloud, ind):
    # Compute normals to help visualize
    # cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    inlier_cloud = cloud.select_by_index(ind)
    inlier_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
obs_env = PCDObsEnv(
    voxel_size=0.002
)
bg = BackgroundGeometry()

Loaded camera alignment for cameras [0, 1, 2, 3]
Started camera 0
Started camera 1
Started camera 2
Started camera 3
:: Background point cloud loaded from /home/bowen/Projects/hacman_real_robot/hacman_real_env/pcd_obs_env/segmentation_params/background.pcd
:: Background params loaded from /home/bowen/Projects/hacman_real_robot/hacman_real_env/pcd_obs_env/segmentation_params/background_params.pkl


In [4]:
obj_name = 'color_block3'
full_pcd = None

## Manual Registration

In [5]:
def pick_points(pcd1, pcd2, tansform1):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    pcd1 = copy.deepcopy(pcd1)
    pcd2 = copy.deepcopy(pcd2)
    pcd1.paint_uniform_color([1, 0.706, 0])
    pcd2.paint_uniform_color([0, 0.651, 0.929])
    pcd1.transform(tansform1)

    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd1 + pcd2)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    picked_points = vis.get_picked_points()
    num_points = len(picked_points)
    assert num_points % 2 == 0, "Must pick an even number of points"

    if num_points == 0:
        return None
    
    picked_points1 = picked_points[:num_points // 2]
    picked_points2 = np.array(picked_points[num_points // 2:]) - len(pcd1.points)
    return picked_points1, picked_points2


def demo_manual_registration(source, target, source_bg=None, target_bg=None):
    print("Demo for manual ICP")
    print("Visualization of two point clouds before manual alignment")
    trans_init = np.identity(4)
    picked_id_source = []
    picked_id_target = []

    while True:
        # pick points from two point clouds and builds correspondences
        picked_ids = pick_points(source, target, trans_init)
        if picked_ids is None:
            break

        picked_id_source.extend(picked_ids[0])
        picked_id_target.extend(picked_ids[1])
        assert (len(picked_id_source) == len(picked_id_target))
        corr = np.zeros((len(picked_id_source), 2))
        corr[:, 0] = picked_id_source
        corr[:, 1] = picked_id_target

        # estimate rough transformation using correspondences
        print("Compute a rough transform using the correspondences given by user")
        p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
        trans_init = p2p.compute_transformation(source, target,
                                                o3d.utility.Vector2iVector(corr))

        # point-to-point ICP for refinement
        print("Perform point-to-point ICP refinement")
        threshold = 0.01  # 1cm distance threshold
        reg_p2p = o3d.pipelines.registration.registration_icp(
            source, target, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            criteria=o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-06, relative_rmse=1e-06, max_iteration=1000))
        draw_registration_result(source, target, reg_p2p.transformation)
        trans_init = reg_p2p.transformation
        print("")
    
    return trans_init


In [7]:
# Record an object pcd
pcd = obs_env.get_pcd(
    return_numpy=False,
    color=True)
pcd, bg_mask = bg.process_pcd(
                    pcd,
                    replace_bg=False,
                    debug=False)
obj_pcd = pcd.select_by_index(bg_mask, invert=True)
bg_pcd = pcd.select_by_index(bg_mask, invert=False)

# icp to the full pcd
if full_pcd is None:
    full_pcd = obj_pcd
else:
    transform = demo_manual_registration(
        obj_pcd, full_pcd, None, None)
    transformed_obj_pcd = obj_pcd.transform(transform)

    full_pcd_ = copy.deepcopy(full_pcd)
    full_pcd_ += transformed_obj_pcd
    full_pcd_ = full_pcd_.voxel_down_sample(
        voxel_size=0.002)
    
    o3d.visualization.draw_geometries([full_pcd_])

Cam 0. Capture takes 0.320s. Processing takes 0.042s.
Cam 1. Capture takes 0.325s. Processing takes 0.031s.
Cam 2. Capture takes 0.326s. Processing takes 0.036s.
Cam 3. Capture takes 0.329s. Processing takes 0.029s.
Obtaining pcds takes 1.446s.
Demo for manual ICP
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] No point has been picked.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #395 (0.045, -0.095, 0.056) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #481 (-0.054, -0.1, 0.031) to add in queue.
[Open3D INFO] Picked point #914 (-0.056, -0.17, 0.013) to add in queue.
[Open3D INFO] Picked point #2105 (0.082, 0.094, 0.043) to add in queue.
[Open3D INFO] Picked point #1845 (0.0087, 0.14, 0.077) to add in queue.
[Open3D INFO] No

In [16]:
# Remove outliers
full_pcd_copy = copy.deepcopy(full_pcd_)
_, idx = full_pcd_copy.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.45)
display_inlier_outlier(full_pcd_copy, idx)

Showing outliers (red) and inliers (gray): 


In [17]:
o3d.visualization.draw_geometries([full_pcd_copy.select_by_index(idx)])

In [18]:
full_pcd = full_pcd_copy.select_by_index(idx)

In [19]:
full_pcd_dir = 'full_pcds'
os.makedirs(full_pcd_dir, exist_ok=True)
filepath = os.path.join(full_pcd_dir, f"{obj_name}.pcd")
o3d.io.write_point_cloud(filepath, full_pcd)

True

In [20]:
obj_name

'orange_car'

libusb: error [udev_hotplug_event] ignoring udev action change
libusb: error [udev_hotplug_event] ignoring udev action bind


In [ ]:
def run_object_icp(source, target, output_fitness=False, visualize=False):
    t_start = time.time()
    # source = source.voxel_down_sample(0.005)
    source.estimate_normals()
    source.orient_normals_consistent_tangent_plane(10)
    obb = source.get_oriented_bounding_box()
    obb.color = (1, 0, 0)
    
    # target = target.voxel_down_sample(0.005)
    target.estimate_normals()
    target.orient_normals_consistent_tangent_plane(10)
    obb_target = target.get_oriented_bounding_box()
    obb_target.color = (0, 1, 0)
    
    # Global
    voxel_size=0.005
    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    icp_result = execute_global_registration(source_down, target_down,
                                                source_fpfh, target_fpfh,
                                                voxel_size)
    t_global = time.time()

    # source_new = copy.deepcopy(source).transform(icp_result)
    # obb_new = source_new.get_oriented_bounding_box()
    # obb_new.color = (0, 1, 0)
    # o3d.visualization.draw_geometries([source, source_new, target, obb, obb_target, obb_new])

    if visualize:
        transform = np.eye(4)
        draw_registration_result(source, target, transform)

    # Local
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance=0.2, init=icp_result,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-06, relative_rmse=1e-06, max_iteration=1000))
    print("ICP: Fitness {:.2f} \t MSE {:.2e}".format(reg_p2p.fitness, reg_p2p.inlier_rmse))
    icp_result = reg_p2p.transformation
    t_local = time.time()
    print("ICP: Global {:.2f} sec. \t Local {:.2e} sec.".format(t_global - t_start, t_local - t_global))

    # # Finetuned local
    # reg_p2p = o3d.pipelines.registration.registration_icp(
    #     source, target, max_correspondence_distance=0.015, init=icp_result,
    #     estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    #     criteria=o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-06, relative_rmse=1e-06, max_iteration=100))
    # print("ICP: Fitness {:.2f} \t MSE {:.2e}".format(reg_p2p.fitness, reg_p2p.inlier_rmse))
    # icp_result = reg_p2p.transformation
    # t_local = time.time()
    # print("ICP: Global {:.2f} sec. \t Local {:.2e} sec.".format(t_global - t_start, t_local - t_global))

    if visualize:
        draw_registration_result(source, target, icp_result)
    
    if reg_p2p.fitness < 0.8:
        print("*"*50)
        print("*** WARNING: ICP fitness too low! ***")
        print("*"*50)
    
    # source_new = copy.deepcopy(source).transform(icp_result)
    # obb_new = source_new.get_oriented_bounding_box()
    # obb_new.color = (0, 0, 1)
    # o3d.visualization.draw_geometries([source, source_new, target, obb, obb_target, obb_new])
    
    if output_fitness:
        return icp_result, reg_p2p.fitness

    return icp_result

In [ ]:
# Record an object pcd
pcd = obs_env.get_pcd(return_numpy=False)
pcd, bg_mask = bg.process_pcd(
                    pcd,
                    replace_bg=False,
                    debug=False)
obj_pcd = pcd.select_by_index(bg_mask, invert=True)

# icp to the full pcd
if full_pcd is None:
    full_pcd = obj_pcd
else:
    transform = run_object_icp(obj_pcd, full_pcd, visualize=True)
    transformed_obj_pcd = obj_pcd.transform(transform)

    full_pcd_ = copy.deepcopy(full_pcd)
    full_pcd_ += transformed_obj_pcd
    full_pcd_ = full_pcd_.voxel_down_sample(voxel_size=0.005)
    
    o3d.visualization.draw_geometries([full_pcd_])

In [ ]:
# Remove outliers
_, idx = full_pcd_.remove_statistical_outlier(nb_neighbors=10, std_ratio=0.03)
display_inlier_outlier(full_pcd_, idx)
o3d.visualization.draw_geometries([full_pcd_.select_by_index(idx)])

In [ ]:
full_pcd_ = full_pcd_.select_by_index(idx)

In [ ]:
full_pcd = full_pcd_

In [ ]:
full_pcd_dir = 'full_pcds'
os.makedirs(full_pcd_dir, exist_ok=True)
filepath = os.path.join(full_pcd_dir, f"{obj_name}.pcd")
o3d.io.write_point_cloud(filepath, full_pcd)